# Features engineer

In [ ]:
import os
import json
import random
import pickle
import pandas as pd
import numpy as np
import xgboost as xgb
from scipy.stats import spearmanr
from datetime import datetime

# ============================================================
#                  GLOBAL REPRODUCIBILITY
# ============================================================
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
print(f"🔒 Global seed set to {SEED}")

# ============================================================
#                  EXPERIMENT CONFIGURATION
# ============================================================
EXPERIMENT_DIR = "../results/xgb/model_setting/v3-best/X2_X1_pruned"
TRAIN_PATH = "../preprocessed_data/CAGE-merged/4. features selection/X2_all_rank_features_pruned.tsv"
VAL_PATH   = "../preprocessed_data/CAGE-merged/4. features selection/X1_all_rank_features_pruned.tsv"

META_COLS = ["gene_name", "chr", "gene_start", "gene_end",
             "TSS_start", "TSS_end", "strand", "gex", "gex_rank"]
TARGET_COL = "gex_rank"

os.makedirs(EXPERIMENT_DIR, exist_ok=True)
LOG_PATH = os.path.join(EXPERIMENT_DIR, "log.txt")

# ✅ 若舊 log.txt 存在則清空
with open(LOG_PATH, "w") as f:
    f.write("")

def log(msg):
    """印出並同步寫入 log.txt"""
    print(msg)
    with open(LOG_PATH, "a") as f:
        f.write(f"{msg}\n")

log(f"🚀 Experiment started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
log(f"Experiment directory: {EXPERIMENT_DIR}")

# ============================================================
#                  DATA LOADING
# ============================================================
df_train_full = pd.read_csv(TRAIN_PATH, sep="\t")
df_val_full   = pd.read_csv(VAL_PATH, sep="\t")

feature_cols = [c for c in df_train_full.columns if c not in META_COLS]
log(f"Feature count: {len(feature_cols)}")

# ============================================================
#                  K-FOLD DEFINITION
# ============================================================
chromosomes = [f"chr{i}" for i in range(2, 23)]
folds = [chromosomes[i::5] for i in range(5)]

log("🧩 Chromosome folds:")
for i, fset in enumerate(folds):
    log(f"Fold {i+1}: {fset}")

# ============================================================
#                  XGBOOST PARAMETERS
# ============================================================
params = {
        "booster": "gbtree",
        "verbosity": 1,
        "learning_rate": 0.016676974956976915,
        "max_depth": 15,
        "colsample_bytree": 0.64561553423692,
        "subsample": 0.8113835038425429,
        "subsample_freq": 6,
        "alpha": 0.3316673054635859,
        "lambda": 0.8969317795206216,
        "gamma": 0.04923442843722911,
        "min_child_weight": 38,
        "max_leaves": 48,
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "nthread": 8,
        "random_state": SEED,
        "tree_method": "hist"
    }
# ============================================================
#                  SAVE CONFIG
# ============================================================
config = {
    "train_path": TRAIN_PATH,
    "val_path": VAL_PATH,
    "seed": SEED,
    "params": params,
    "folds": folds,
    "target_col": TARGET_COL,
}
with open(os.path.join(EXPERIMENT_DIR, "config.json"), "w") as f:
    json.dump(config, f, indent=4)

# ============================================================
#                  TRAINING LOOP
# ============================================================
results = []
preds_all = []

for fold_idx, val_chrs in enumerate(folds):
    log(f"\n🚀 Fold {fold_idx+1}/{len(folds)} | Val chromosomes: {val_chrs}")
    fold_dir = os.path.join(EXPERIMENT_DIR, f"fold_{fold_idx+1}")
    os.makedirs(fold_dir, exist_ok=True)

    # Split
    train_chrs = [c for c in chromosomes if c not in val_chrs]
    df_train = df_train_full[df_train_full["chr"].isin(train_chrs)].copy()
    df_val   = df_val_full[df_val_full["chr"].isin(val_chrs)].copy()

    df_val = df_val.reset_index().rename(columns={"index": "orig_idx"})

    X_train, y_train = df_train[feature_cols], df_train[TARGET_COL]
    X_val, y_val     = df_val[feature_cols], df_val[TARGET_COL]

    # Create DMatrix
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval   = xgb.DMatrix(X_val, label=y_val)

    # Train
    model = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=2000,
        evals=[(dtrain, "train"), (dval, "val")],
        early_stopping_rounds=100,
        verbose_eval=200
    )

    # Save model (.json)
    model_path = os.path.join(fold_dir, "model.json")
    model.save_model(model_path)
    log(f"💾 Model saved: {model_path}")

    # Predict
    df_val["predicted_gex_rank"] = model.predict(dval, iteration_range=(0, model.best_iteration))
    df_val["fold_id"] = fold_idx + 1
    preds_all.append(df_val[["orig_idx", "gene_name", "predicted_gex_rank", "fold_id"]])

    # Evaluate Spearman
    overall_spearman = spearmanr(df_val[TARGET_COL], df_val["predicted_gex_rank"])[0]
    log(f"📈 Fold {fold_idx+1} Overall Spearman ρ = {overall_spearman:.4f}")

    log("📊 Per-Chromosome Spearman:")
    chr_corrs = []
    for chrom, subdf in df_val.groupby("chr"):
        if len(subdf) < 2:
            continue
        rho = spearmanr(subdf[TARGET_COL], subdf["predicted_gex_rank"])[0]
        chr_corrs.append({"chr": chrom, "spearman": rho})
        log(f"   {chrom:<6s}: ρ = {rho:.4f}")

    results.append({
        "fold": fold_idx + 1,
        "val_chr": val_chrs,
        "overall_spearman": overall_spearman,
        "per_chr": chr_corrs
    })

# ============================================================
#                  SAVE PREDICTIONS & SUMMARY
# ============================================================
df_preds_all = pd.concat(preds_all, ignore_index=True)
df_preds_all = df_preds_all.sort_values("orig_idx").reset_index(drop=True)

assert len(df_preds_all) == len(df_val_full)
assert (df_preds_all["gene_name"].values == df_val_full["gene_name"].values).all(), \
    "❌ Prediction order mismatch with df_val_full!"

df_preds_all[["gene_name", "predicted_gex_rank"]].to_csv(
    os.path.join(EXPERIMENT_DIR, "predict_val.tsv"), sep="\t", index=False
)
log("💾 Final predictions saved (predict_val.tsv)")

# Evaluate overall alignment
overall_val_spearman = spearmanr(
    df_val_full["gex_rank"],
    df_preds_all["predicted_gex_rank"]
)[0]
log(f"🌍 Overall validation Spearman (aligned) = {overall_val_spearman:.4f}")

# Summaries
summary_fold = pd.DataFrame([
    {"fold": r["fold"], "val_chr": ",".join(r["val_chr"]), "overall_spearman": r["overall_spearman"]}
    for r in results
])
chr_rows = []
for r in results:
    for c in r["per_chr"]:
        chr_rows.append({"fold": r["fold"], "chr": c["chr"], "spearman": c["spearman"]})
summary_chr = pd.DataFrame(chr_rows)
chr_mean = summary_chr.groupby("chr", as_index=False)["spearman"].mean()
chr_mean.rename(columns={"spearman": "mean_spearman"}, inplace=True)

log(f"Mean Spearman (across folds): {summary_fold['overall_spearman'].mean():.4f}")
log(f"Overall mean of per-chromosome means: {chr_mean['mean_spearman'].mean():.4f}")

log(f"\n✅ Experiment finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


🔒 Global seed set to 42
🚀 Experiment started at 2025-10-14 12:04:14
Experiment directory: ../results/xgb_linear/model_setting/baseline
Feature count: 277
🧩 Chromosome folds:
Fold 1: ['chr2', 'chr7', 'chr12', 'chr17', 'chr22']
Fold 2: ['chr3', 'chr8', 'chr13', 'chr18']
Fold 3: ['chr4', 'chr9', 'chr14', 'chr19']
Fold 4: ['chr5', 'chr10', 'chr15', 'chr20']
Fold 5: ['chr6', 'chr11', 'chr16', 'chr21']

🚀 Fold 1/5 | Val chromosomes: ['chr2', 'chr7', 'chr12', 'chr17', 'chr22']
[0]	train-rmse:0.26945	val-rmse:0.26053
[133]	train-rmse:0.25155	val-rmse:0.26140
💾 Model saved: ../results/xgb_linear/model_setting/baseline\fold_1\model.json
📈 Fold 1 Overall Spearman ρ = 0.5936
📊 Per-Chromosome Spearman:
   chr12 : ρ = 0.6198
   chr17 : ρ = 0.5674
   chr2  : ρ = 0.6045
   chr22 : ρ = 0.6143
   chr7  : ρ = 0.5740

🚀 Fold 2/5 | Val chromosomes: ['chr3', 'chr8', 'chr13', 'chr18']
[0]	train-rmse:0.27016	val-rmse:0.25730
[200]	train-rmse:0.25463	val-rmse:0.23900
[400]	train-rmse:0.25422	val-rmse:0.23850
[

In [ ]:
import optuna
import xgboost as xgb
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import warnings, json, os, random

warnings.filterwarnings("ignore")

# ============================================================
#                  CONFIGURATION
# ============================================================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

TRAIN_PATH = "../preprocessed_data/CAGE-merged/4. features selection/X1_all_rank_features_pruned.tsv"
VAL_PATH   = "../preprocessed_data/CAGE-merged/4. features selection/X2_all_rank_features_pruned.tsv"

META_COLS = [
    "gene_name", "chr", "gene_start", "gene_end",
    "TSS_start", "TSS_end", "strand", "gex", "gex_rank"
]
TARGET_COL = "gex_rank"

# ============================================================
#                  LOAD DATA
# ============================================================
df_train = pd.read_csv(TRAIN_PATH, sep="\t")
df_val   = pd.read_csv(VAL_PATH, sep="\t")

# 合併為整體資料池（用 chromosome 抽 fold）
df_all = pd.concat([df_train, df_val], ignore_index=True)
chromosomes = sorted(df_all["chr"].unique())

print(f"🧬 Total chromosomes in dataset: {chromosomes}")
print(f"Total samples: {len(df_all):,}")

feature_cols = [c for c in df_all.columns if c not in META_COLS]

# ============================================================
#                  OBJECTIVE FUNCTION
# ============================================================
def objective(trial):
    """Optuna optimization objective for XGBoost hyperparameters"""

    # 1️⃣ 固定 chr6, chr13 + 隨機抽另外兩條作為 validation
    fixed_val_chrs = ["chr6", "chr13"]
    remaining_chrs = [c for c in chromosomes if c not in fixed_val_chrs]
    rng = random.Random(SEED)  # 每個 trial 有不同抽樣種子
    random_extra = rng.sample(remaining_chrs, 2)
    val_chrs = fixed_val_chrs + random_extra
    print(f"Validation chromosomes for this trial: {val_chrs}")
    train_chrs = [c for c in chromosomes if c not in val_chrs]

    df_train_fold = df_all[df_all["chr"].isin(train_chrs)].copy()
    df_val_fold   = df_all[df_all["chr"].isin(val_chrs)].copy()

    X_train, y_train = df_train_fold[feature_cols], df_train_fold[TARGET_COL]
    X_val, y_val     = df_val_fold[feature_cols], df_val_fold[TARGET_COL]

    # ============================================================
    #                  PARAMETER SEARCH SPACE
    # ============================================================
    params = {
        "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
        "objective": "reg:squarederror",
        "eval_metric": trial.suggest_categorical("booeval_metricster", ["rmse", "mae"]),
        "verbosity": 0,
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.05, log=True),
        "max_depth": trial.suggest_int("max_depth", 4, 10),
        "max_leaves": trial.suggest_int("max_leaves", 16, 128, step=8),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "alpha": trial.suggest_float("alpha", 0.0, 1.0),      # L1
        "lambda": trial.suggest_float("lambda", 0.0, 2.0),    # L2
        "gamma": trial.suggest_float("gamma", 0.0, 0.05),     # min_gain_to_split
        "min_child_weight": trial.suggest_int("min_child_weight", 10, 100),
        "tree_method": "hist",
        "seed": SEED,
        "nthread": 8,
    }

    # ============================================================
    #                  TRAINING
    # ============================================================
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval   = xgb.DMatrix(X_val, label=y_val)

    model = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=300,
        evals=[(dval, "val")],
        early_stopping_rounds=100,
        verbose_eval=False
    )

    # ============================================================
    #                  EVALUATION
    # ============================================================
    y_pred = model.predict(dval, iteration_range=(0, model.best_iteration))
    rho_overall = spearmanr(y_val, y_pred)[0]

    # 每個 chromosome 的 Spearman
    per_chr_rho = []
    for chrom, subdf in df_val_fold.assign(pred=y_pred).groupby("chr"):
        if len(subdf) > 2:
            per_chr_rho.append(spearmanr(subdf["gex_rank"], subdf["pred"])[0])
    mean_rho = np.nanmean(per_chr_rho)

    # 可選：回傳整體或平均 Spearman
    return mean_rho if not np.isnan(mean_rho) else rho_overall


# ============================================================
#                  RUN OPTIMIZATION
# ============================================================
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=SEED)
)
study.optimize(objective, n_trials=300, show_progress_bar=True)

# ============================================================
#                  SAVE RESULTS
# ============================================================
print("✅ Best Spearman:", study.best_value)
print("🏆 Best Parameters:")
for k, v in study.best_params.items():
    print(f"   {k}: {v}")

SAVE_DIR = "../results/xgb/optuna_fixed_chr6_chr13"
os.makedirs(SAVE_DIR, exist_ok=True)

study.trials_dataframe().to_csv(os.path.join(SAVE_DIR, "trials_log.csv"), index=False)
with open(os.path.join(SAVE_DIR, "best_params.json"), "w") as f:
    json.dump(study.best_params, f, indent=4)

print(f"💾 Saved best params and log to {SAVE_DIR}")


[I 2025-10-14 03:09:07,788] A new study created in memory with name: no-name-dec0f6f7-7094-40f6-a3f8-fa0a0e29d26e


🧬 Total chromosomes in dataset: ['chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr2', 'chr20', 'chr21', 'chr22', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9']
Total samples: 32,568


  0%|          | 0/300 [00:00<?, ?it/s]

Validation chromosomes for this trial: ['chr6', 'chr13', 'chr14', 'chr10']


Best trial: 0. Best value: 0.740786:   0%|          | 1/300 [01:32<7:42:38, 92.84s/it]

...

[I 2025-10-14 04:34:55,442] Trial 297 finished with value: 0.7557597060754432 and parameters: {'booster': 'gbtree', 'booeval_metricster': 'rmse', 'learning_rate': 0.027083663697253877, 'max_depth': 5, 'max_leaves': 104, 'subsample': 0.9572318696832974, 'colsample_bytree': 0.6219868741196499, 'alpha': 0.4503633754423479, 'lambda': 1.5262641275400521, 'gamma': 0.002292296871210129, 'min_child_weight': 47}. Best is trial 104 with value: 0.7638603446499435.
Validation chromosomes for this trial: ['chr6', 'chr13', 'chr14', 'chr10']


Best trial: 104. Best value: 0.76386: 100%|█████████▉| 299/300 [1:25:57<00:21, 21.47s/it]

[I 2025-10-14 04:35:05,223] Trial 298 finished with value: 0.7619986532812137 and parameters: {'booster': 'gbtree', 'booeval_metricster': 'rmse', 'learning_rate': 0.04519446595613783, 'max_depth': 10, 'max_leaves': 80, 'subsample': 0.9134876663721249, 'colsample_bytree': 0.7015847455831823, 'alpha': 0.15604833412918565, 'lambda': 1.0358050848737026, 'gamma': 0.0042849978988866895, 'min_child_weight': 54}. Best is trial 104 with value: 0.7638603446499435.
Validation chromosomes for this trial: ['chr6', 'chr13', 'chr14', 'chr10']


Best trial: 104. Best value: 0.76386: 100%|██████████| 300/300 [1:26:08<00:00, 17.23s/it]

[I 2025-10-14 04:35:16,642] Trial 299 finished with value: 0.7615009027395874 and parameters: {'booster': 'gbtree', 'booeval_metricster': 'rmse', 'learning_rate': 0.037331240950264126, 'max_depth': 10, 'max_leaves': 96, 'subsample': 0.8993319007332586, 'colsample_bytree': 0.6494463343645109, 'alpha': 0.378505465088032, 'lambda': 0.6264322198559682, 'gamma': 0.00113732547570053, 'min_child_weight': 42}. Best is trial 104 with value: 0.7638603446499435.
✅ Best Spearman: 0.7638603446499435
🏆 Best Parameters:
   booster: gbtree
   booeval_metricster: mae
   learning_rate: 0.035962471748043
   max_depth: 10
   max_leaves: 104
   subsample: 0.8971619038015882
   colsample_bytree: 0.6636371114832225
   alpha: 0.12734537736853208
   lambda: 1.628472451981166
   gamma: 0.0024413287883050493
   min_child_weight: 53
💾 Saved best params and log to ../results/xgb/optuna_fixed_chr6_chr13
